In [1]:
import pandas as pd
import numpy as np
import glob

In [42]:
filteredGeneList = pd.read_csv('../RBC/Database/filteredGenesDetails_human_240118.txt',sep='\t')
acc2gene={}
gene2acc={}
for idx,row in filteredGeneList.iterrows():
    acc2gene[row['AccNum']] = row['GeneName']
    gene2acc[row['GeneName']] = row['AccNum']
filteredGeneList.head(2)

,GeneName,AccNum,Chrom,mRNALength,ORFLength,fpUTR_length,tpUTR_length,sumIntronLengths
0,UBE2Q1,NM_017582,chr1,3223,1269,91,1863,6847
1,DUOXA1,NM_001276267,chr15,3032,897,395,1740,9479


In [43]:
TIS = pd.read_csv('../RBC/Database/accnum2tis.csv')
TIS.columns = ['AccNum','TIS']
TIS.head()

,AccNum,TIS
0,NM_017582,101.0
1,NM_001276267,111.0
2,NM_173469,79.0
3,NM_001330474,34.0
4,NM_014372,102.0


In [44]:
logL_L24 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_L24_t1_enriched_likelihood.csv')
logL_S15_t0 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_S15_t0_enriched_likelihood.csv')
logL_S15_t1 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_S15_t1_enriched_likelihood.csv')
logL_S15_t2 = pd.read_csv('../RBC/MOTIF/SeqLogo/gene_likelihood/ER_S15_t2_enriched_likelihood.csv')
logL_L24.head(2)

,Unnamed: 0,logLikelihood,Gene,AccNum
0,0,-14.542486,MYO1D,NM_015194
1,1,-14.550683,PPP1R14B,NM_138689


In [45]:
mfe_fp = pd.read_csv('../RBC/MOTIF/rnafold/fpUTR_minE_normbyLen_fillmean.csv')
mfe_fp_orf = pd.read_csv('../RBC/MOTIF/rnafold/utr_and_orf_minE_normbyLen.csv')
mfe_fp.columns = ['AccNum','MFE_fpUTR_normed']
mfe_fp_orf.columns = ['AccNum','MFE_fpUTRandORF_normed']

In [46]:
comb = pd.DataFrame(logL_L24.loc[:,'Gene'])
print(len(comb))
comb['AccNum'] = logL_L24['AccNum']
comb['seqlogo_logL_L24_t1'] = logL_L24['logLikelihood']
comb['seqlogo_logL_S15_t0'] = logL_S15_t0['logLikelihood']
comb['seqlogo_logL_S15_t1'] = logL_S15_t1['logLikelihood']
comb['seqlogo_logL_S15_t2'] = logL_S15_t2['logLikelihood']
comb = pd.merge(comb, TIS, how='inner', on=['AccNum'])
comb = pd.merge(comb, mfe_fp, how='inner', on=['AccNum'])
comb = pd.merge(comb, mfe_fp_orf, how='inner', on=['AccNum'])

7890


In [47]:
comb

,Gene,AccNum,seqlogo_logL_L24_t1,seqlogo_logL_S15_t0,seqlogo_logL_S15_t1,seqlogo_logL_S15_t2,TIS,MFE_fpUTR_normed,MFE_fpUTRandORF_normed
0,MYO1D,NM_015194,-14.542486,-15.117326,-15.447782,-13.241358,91.0,-0.522658,-0.504090
1,PPP1R14B,NM_138689,-14.550683,-15.397989,-15.667537,-13.566780,91.0,-0.644104,-0.622038
2,AGFG1,NM_001135187,-14.663012,-15.412761,-15.725220,-13.634400,113.0,-0.666440,-0.598229
3,LAMP1,NM_005561,-14.668466,-15.438871,-15.744788,-13.810126,98.0,-0.519715,-0.509029
4,AFG3L2,NM_006796,-14.691183,-15.474773,-15.753664,-13.876347,113.0,-0.560619,-0.620646
...,...,...,...,...,...,...,...,...,...
7885,SDCBP,NM_001348341,-33.683685,-29.864479,-28.686124,-32.418184,44.0,-0.312993,-0.308519
7886,RPRD1A,NM_001303411,-33.846032,-29.997533,-28.854125,-32.556170,79.0,-0.464314,-0.414923
7887,FASTKD1,NM_001322046,-33.846032,-30.293971,-29.278236,-33.208495,73.0,-0.330131,-0.329384
7888,SCRN3,NM_024583,-33.975719,-31.335351,-29.994056,-33.585789,67.0,-0.305612,-0.258283


In [48]:
#comb.to_csv('collate_genes.csv')

In [49]:
ER_files = glob.glob('../RBC/Deseq2 Analysis/shift15/coding/csv_raw/ER*.csv')
deseq=pd.read_csv(ER_files[0])
df=pd.DataFrame({'AccNum':deseq['Unnamed: 0']})

In [50]:
ER_files[1:6]

['../RBC/Deseq2 Analysis/shift15/coding/csv_raw\\ER_L24_t1.csv',
 '../RBC/Deseq2 Analysis/shift15/coding/csv_raw\\ER_L24_t2.csv',
 '../RBC/Deseq2 Analysis/shift15/coding/csv_raw\\ER_S15_t0.csv',
 '../RBC/Deseq2 Analysis/shift15/coding/csv_raw\\ER_S15_t1.csv',
 '../RBC/Deseq2 Analysis/shift15/coding/csv_raw\\ER_S15_t2.csv']

In [51]:
for ER_file in ER_files[1:6]:
    keyword = ER_file.split('\\')[-1][:-4]
    if keyword == 'ER_L24_t2': continue
    deseq = pd.read_csv(ER_file)
    df[keyword+'_logFC'] = deseq['log2FoldChange']
    df[keyword+'_change'] = 'No'
    df.loc[df[keyword+'_logFC']>1,keyword+'_change'] = 'enrich'
    df.loc[df[keyword+'_logFC']<-1,keyword+'_change'] = 'deplete'

In [52]:
comb=pd.merge(comb, df, how='right', on=['AccNum'])

In [53]:
comb

,Gene,AccNum,seqlogo_logL_L24_t1,seqlogo_logL_S15_t0,seqlogo_logL_S15_t1,seqlogo_logL_S15_t2,TIS,MFE_fpUTR_normed,MFE_fpUTRandORF_normed,ER_L24_t1_logFC,ER_L24_t1_change,ER_S15_t0_logFC,ER_S15_t0_change,ER_S15_t1_logFC,ER_S15_t1_change,ER_S15_t2_logFC,ER_S15_t2_change
0,MYO1D,NM_015194,-14.542486,-15.117326,-15.447782,-13.241358,91.0,-0.522658,-0.504090,-0.489914,No,-0.303135,No,-0.466139,No,-0.485069,No
1,PPP1R14B,NM_138689,-14.550683,-15.397989,-15.667537,-13.566780,91.0,-0.644104,-0.622038,1.073262,enrich,1.095194,enrich,1.694895,enrich,0.655761,No
2,AGFG1,NM_001135187,-14.663012,-15.412761,-15.725220,-13.634400,113.0,-0.666440,-0.598229,-0.198402,No,0.158027,No,0.374909,No,0.282730,No
3,LAMP1,NM_005561,-14.668466,-15.438871,-15.744788,-13.810126,98.0,-0.519715,-0.509029,0.290764,No,-0.145868,No,-0.270792,No,-0.268401,No
4,AFG3L2,NM_006796,-14.691183,-15.474773,-15.753664,-13.876347,113.0,-0.560619,-0.620646,-0.085239,No,-0.304327,No,-0.431552,No,-0.262779,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,NaN,NM_001080398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.312778,No,-0.140486,No,-0.174372,No,-0.250475,No
7986,NaN,NM_001170330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072176,No,1.260953,enrich,1.604759,enrich,0.775510,No
7987,NaN,NM_001308632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179873,No,0.280138,No,0.161024,No,-0.197482,No
7988,NaN,NM_001105570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242662,No,0.582128,No,0.184474,No,-0.056924,No


In [54]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d
splicedGenes = get_fa('../RBC/Database/splicedGenes_240118.txt')
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357_chr1(+):11868-14361', 'NR_046018_chr1(+):11873-14408', 'NR_024540_chr1(-):14361-29369']


In [55]:
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
print(type(splicedGenes))
print(list(splicedGenes.keys())[0:3])

<class 'dict'>
['NR_148357', 'NR_046018', 'NR_024540']


In [56]:
splicedORFs = get_fa('../RBC/Database/splicedORFs_240118.txt')
print(type(splicedORFs))
print(list(splicedORFs.keys())[0:3])

<class 'dict'>
['NM_017582', 'NM_001276267', 'NM_173469']


In [57]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [58]:
import re
def countGC(seq):
    if len(seq)==0: return float('nan')
    gc_all_percent = (seq.count('G')+seq.count('C'))/len(seq)
    #codons = (re.findall('...?', seq))
    #codon_tr=[''.join(s) for s in zip(*codons)]
    #gc_last_nt = (codon_tr[2].count('G')+codon_tr[2].count('C'))/len(codons)
    #return gc_all_percent#, 
    return gc_all_percent

In [59]:
fpUTR_gc=[]
for accNum in comb['AccNum']:
    gc = countGC(fpUTR[accNum])
    fpUTR_gc.append(gc)

In [60]:
comb['fpUTR_gc_percent'] = fpUTR_gc

In [61]:
comb.to_csv('collate_genes_20may.csv',index=False)

In [62]:
comb

,Gene,AccNum,seqlogo_logL_L24_t1,seqlogo_logL_S15_t0,seqlogo_logL_S15_t1,seqlogo_logL_S15_t2,TIS,MFE_fpUTR_normed,MFE_fpUTRandORF_normed,ER_L24_t1_logFC,ER_L24_t1_change,ER_S15_t0_logFC,ER_S15_t0_change,ER_S15_t1_logFC,ER_S15_t1_change,ER_S15_t2_logFC,ER_S15_t2_change,fpUTR_gc_percent
0,MYO1D,NM_015194,-14.542486,-15.117326,-15.447782,-13.241358,91.0,-0.522658,-0.504090,-0.489914,No,-0.303135,No,-0.466139,No,-0.485069,No,0.807309
1,PPP1R14B,NM_138689,-14.550683,-15.397989,-15.667537,-13.566780,91.0,-0.644104,-0.622038,1.073262,enrich,1.095194,enrich,1.694895,enrich,0.655761,No,0.828358
2,AGFG1,NM_001135187,-14.663012,-15.412761,-15.725220,-13.634400,113.0,-0.666440,-0.598229,-0.198402,No,0.158027,No,0.374909,No,0.282730,No,0.832000
3,LAMP1,NM_005561,-14.668466,-15.438871,-15.744788,-13.810126,98.0,-0.519715,-0.509029,0.290764,No,-0.145868,No,-0.270792,No,-0.268401,No,0.786477
4,AFG3L2,NM_006796,-14.691183,-15.474773,-15.753664,-13.876347,113.0,-0.560619,-0.620646,-0.085239,No,-0.304327,No,-0.431552,No,-0.262779,No,0.798969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7985,NaN,NM_001080398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.312778,No,-0.140486,No,-0.174372,No,-0.250475,No,NaN
7986,NaN,NM_001170330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072176,No,1.260953,enrich,1.604759,enrich,0.775510,No,NaN
7987,NaN,NM_001308632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179873,No,0.280138,No,0.161024,No,-0.197482,No,NaN
7988,NaN,NM_001105570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242662,No,0.582128,No,0.184474,No,-0.056924,No,NaN


In [63]:
def get_fa(filename,onlyKeys = 'All'):
    keyD = dict()
    if onlyKeys!='All':
        for k in onlyKeys: keyD[k] = None
        
    def addEntry(current,d):
        entry = current.split('\n')
        if entry[0]!='' and (onlyKeys=='All' or keyD.has_key(entry[0])):
            d[entry[0]] = ''.join(entry[1:])

    d = dict()
    bf = open(filename)
    current = ['\n']
    for line in bf:
        if line[0]=='>':
            addEntry(''.join(current),d)
            if len(line)>1: current = [line[1:]]
            else: current = ['\n']
        else: current.append(line)
    addEntry(''.join(current),d)
    bf.close()
    return d

In [64]:
splicedGenes = get_fa('../RBC/Database/splicedGenes_240118.txt')
## truncate the keys to only accNum
ks = list(splicedGenes.keys())
for key in ks:
    splicedGenes[key.split('_chr')[0]] = splicedGenes[key]
    splicedGenes.pop(key)
splicedORFs = get_fa('../RBC/Database/splicedORFs_240118.txt')

In [65]:
fpUTR={}
tpUTR = {}
ORFs = {}
for idx,row in filteredGeneList.iterrows():
    accNum = row['AccNum']
    fpLen = row['fpUTR_length']
    tpLen = row['tpUTR_length']
    fpUTR[accNum] = splicedGenes[accNum][0:fpLen]
    tpUTR[accNum] = splicedGenes[accNum][-tpLen:]
    ORFs[accNum] = splicedORFs[accNum]

In [66]:
def gene2seq(acc):
    fp=fpUTR[acc]
    if len(fp)<8:
        return 'NaN'
    seq = fp[-8:]+ORFs[acc][:11]
    #print('# of valid seqs: ',len(seqs))
    return seq

In [69]:
comb['SeqLogo'] = [gene2seq(acc) for acc in comb['AccNum']]

In [71]:
comb.to_csv('collate_genes_with_seqlogo_seq.csv',index=False)